### dynamoDB
This notebook adds data to the dynamoDB database

In [2]:
import boto3
import os
import json
import csv
from dotenv import load_dotenv
import numpy as np
import pandas as pd

In [3]:
# Load the environment variables from the .env file
load_dotenv()

True

In [4]:
# Creating the DynamoDB Client
# make sure to set the AWS_ACCESS_KEY_ID and
# AWS_SECRET_ACCESS_KEY environment variables
# OR pass credentials when creating the client:
dynamodb_client = boto3.client('dynamodb', region_name="us-east-1")

TABLE_NAME = "fumehoods"

dyn_resource = boto3.resource("dynamodb", region_name="us-east-1")
TABLE_NAME = "fumehoods"
table = dyn_resource.Table(TABLE_NAME)

In [9]:
names_df = pd.read_csv("namingconvention.csv", index_col=0)
names_df = names_df.rename({"Floor":"floor", "Lab":"lab", 
                            "Flow Sensor":"flow_sensor","Sash":"sash_sensor",
                            "Occupancy":"occ_sensor",
                            "Internal Temp": "internal_temp_sensor", "External Temp": "external_temp_sensor"}, axis=1)
names_df['name'] = names_df['building'].str.capitalize() + \
                        ".Floor_" + names_df['floor'].astype(str) + \
                        ".Lab_" + names_df['lab'] + \
                        ".Hood_" + names_df['hood'].astype(str)
names_df

,name,building,floor,lab,hood,Server,flow_sensor,sash_sensor,occ_sensor,internal_temp_sensor,external_temp_sensor,FloorNum,Energy Occupied,Energy Unoccupied,Sash Open Time Occupied,Sash Open Time Unoccupied
0,Biotech.Floor_G.Lab_G54.Hood_1,biotech,G,G54,1,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_G54.Hood_1.energy.occ,Biotech.nanFloor.Lab_G54.Hood_1.energy.unocc,Biotech.nanFloor.Lab_G54.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_G54.Hood_1.sashOpenTime.u...
1,Biotech.Floor_1.Lab_141.Hood_1,biotech,1,141,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_141.Hood_1.energy.occ,Biotech.nanFloor.Lab_141.Hood_1.energy.unocc,Biotech.nanFloor.Lab_141.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_141.Hood_1.sashOpenTime.u...
2,Biotech.Floor_1.Lab_143.Hood_1,biotech,1,143,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_143.Hood_1.energy.occ,Biotech.nanFloor.Lab_143.Hood_1.energy.unocc,Biotech.nanFloor.Lab_143.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_143.Hood_1.sashOpenTime.u...
3,Biotech.Floor_1.Lab_147B.Hood_1,biotech,1,147B,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_147B.Hood_1.energy.occ,Biotech.nanFloor.Lab_147B.Hood_1.energy.unocc,Biotech.nanFloor.Lab_147B.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_147B.Hood_1.sashOpenTime....
4,Biotech.Floor_1.Lab_153.Hood_1,biotech,1,153,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_153.Hood_1.energy.occ,Biotech.nanFloor.Lab_153.Hood_1.energy.unocc,Biotech.nanFloor.Lab_153.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_153.Hood_1.sashOpenTime.u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Biotech.Floor_4.Lab_445B.Hood_1,biotech,4,445B,1,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_445B.Hood_1.energy.occ,Biotech.nanFloor.Lab_445B.Hood_1.energy.unocc,Biotech.nanFloor.Lab_445B.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_445B.Hood_1.sashOpenTime....
62,Biotech.Floor_4.Lab_453.Hood_1,biotech,4,453,1,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_453.Hood_1.energy.occ,Biotech.nanFloor.Lab_453.Hood_1.energy.unocc,Biotech.nanFloor.Lab_453.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_453.Hood_1.sashOpenTime.u...
63,Biotech.Floor_4.Lab_453B.Hood_1,biotech,4,453B,1,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/

In [10]:
def format_item(row):
    formatted_row = {}
    for key, value in row.items():
        formatted_row[key] = {'S': str(value)}
    return {'M':formatted_row}

def get_hoods_dict(hoods_columns = ["building", "floor", "lab", "hood", "flow_sensor","sash_sensor"]):
    hoods_names_df = names_df.set_index('name')[hoods_columns]

    hoods_data = hoods_names_df.apply(format_item, axis=1).to_dict()

    hoods = {
                "id": {"S":"hoods"},
                "map": {"M":hoods_data}
    }

    return hoods

def get_buildings_dict(buildings_columns = ["external_temp_sensor"]):
    buildings_names_df = names_df.copy().groupby('building').first().reset_index()
    buildings_names_df['building_name'] = buildings_names_df['building'].str.capitalize()
    buildings_names_df = buildings_names_df.set_index('building_name')[buildings_columns]

    buildings_data = buildings_names_df.apply(format_item, axis=1).to_dict()

    buildings = {
                "id": {"S":"buildings"},
                "map": {"M":buildings_data}
    }

    return buildings

def get_labs_dict(labs_columns = ["building", "floor", "lab", "occ_sensor", "internal_temp_sensor"]):
    labs_names_df = names_df.copy().groupby(['building','floor','lab']).first().reset_index()
    labs_names_df['lab_name'] = names_df['building'].str.capitalize() + \
                                        ".Floor_" + names_df['floor'].astype(str) + \
                                        ".Lab_" + names_df['lab']
    labs_names_df = labs_names_df.set_index('lab_name')[labs_columns]

    labs = {
        "id": "labs",
        "map": labs_names_df.to_dict('index')
    }

    return labs

In [17]:
get_labs_dict()

{'id': 'labs',
 'map': {'Biotech.Floor_G.Lab_G54': {'building': 'biotech',
   'floor': '1',
   'lab': '141',
   'occ_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_141_control/occ',
   'internal_temp_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_141_control/zone/zone_temp/trend_log'},
  'Biotech.Floor_1.Lab_141': {'building': 'biotech',
   'floor': '1',
   'lab': '143',
   'occ_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_143_control/occ',
   'internal_temp_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_143_control/zone/zone_temp/trend_log'},
  'Biotech.Floor_1.Lab_143': {'building': 'biotech',
   'floor': '1',
   'lab': '147B',
   'occ_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_147b_control/occ',
   'internal_temp_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_147b_control/zone/zone_temp/trend_log'},
  'Biotech.Floor_1

In [11]:
response = dynamodb_client.describe_table(TableName=TABLE_NAME)
display(response)

{'Table': {'AttributeDefinitions': [{'AttributeName': 'id',
    'AttributeType': 'S'}],
  'TableName': 'fumehoods',
  'KeySchema': [{'AttributeName': 'id', 'KeyType': 'HASH'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2023, 5, 5, 7, 15, 46, 422000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(2024, 2, 10, 12, 56, 41, 399000, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(2024, 2, 10, 13, 12, 30, 607000, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 1,
   'WriteCapacityUnits': 1},
  'TableSizeBytes': 20443,
  'ItemCount': 11,
  'TableArn': 'arn:aws:dynamodb:us-east-1:498547149247:table/fumehoods',
  'TableId': 'eb54825d-1e24-4311-9ad1-6d0348f6616c',
  'TableClassSummary': {'TableClass': 'STANDARD'},
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'BFAIH178NVHNLHT0P56R71KH2NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'se

In [ ]:
dynamodb_client.

In [16]:
with table.batch_writer() as writer:
    writer.put_item(Item=get_labs_dict())

response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "Biotech.Floor_4.Lab_433.Hood_1"}}
)
print("Labs:")
display(response["Item"])

Labs:


{'id': {'S': 'Biotech.Floor_4.Lab_433.Hood_1'},
 'M': {'M': {'flow_sensor': {'S': '#path'}}}}

In [15]:
response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "labs"}}
)
display(response['Item'])

{'id': {'S': 'labs'},
 'map': {'M': {'Biotech.Floor_4.Lab_417B': {'M': {'occ_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_418_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_418_control/zone/zone_temp/trend_log'},
     'floor': {'S': '4'},
     'lab': {'S': '418'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_3.Lab_327': {'M': {'occ_sensor': {'S': '#biotech/biotech_3rd_floor/third_floor_fume_hood_lab_spaces/lab_327b_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_3rd_floor/third_floor_fume_hood_lab_spaces/lab_327b_control/zone/zone_temp/trend_log'},
     'floor': {'S': '3'},
     'lab': {'S': '327B'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_4.Lab_409B': {'M': {'occ_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_f

In [ ]:
# read Robbie's csv output here
with open('namingconvention.csv', 'r') as file:
    csv_dict_reader = csv.DictReader(file)
    for row in csv_dict_reader:


        hoods = {
            "id": "hoods",
            "map": {
                row['name']: { #"Biotech.Floor_3.Lab_317.Hood_1"
                    "building": row['building'], #"Biotech"
                    "floor": row['Floor'], #"3"
                    "lab": row['Lab'], # "317"
                    "hood": row['hood'], # "1"
                    "flow_sensor": row['Flow Sensor'], # "#path_to_this_sensor"
                    "sash_sensor": row['Sash'], # "#path_to_this_sensor"
                },
                # "Biotech.Floor_4.Lab_433.Hood_1": {
                #     "building": "Biotech",
                #     "floor": "4",
                #     "lab": "433",
                #     "hood": "1",
                #     "flow_sensor": "#path_to_this_sensor",
                #     "sash_sensor": "#path_to_this_sensor",
                # },
            },
        }

        buildings = {
            "id": "buildings",
            "map": {
                # "Biotech": {"external_temp_sensor": "#path_to..."},
                # "Weill": {"external_temp_sensor": "#path_to..."},
                row['name']: {"external_temp_sensor": row['External Temp']}
            },
        }

        labs = {
            "id": "labs",
            "map": {
                row['name']: { #"Biotech.Floor_3.Lab_317"
                    "building": row['building'], #"Biotech"
                    "floor": row['Floor'], # "3"
                    "lab": row['Lab'], # "317"
                    "occ_sensor": row['Occupancy'], # "#path_to_this_sensor"
                    "internal_temp_sensor": row['Internal Temp'], # "#path_to_this_sensor"
                },
            },
        }

        occupants = {
            "id": "occupants",
            "map": {
                "rf377@cornell.edu": {
                    "name": "Reid Fleishman",
                    "lab": "Biotech.Floor_3.Lab_317",
                },
            },
        }

        # Put the above data into the table
        with table.batch_writer() as writer:
            response = writer.put_item(Item=hoods)
            print(response)
            response = writer.put_item(Item=buildings)
            print(response)
            response = writer.put_item(Item=labs)
            print(response)
            response = writer.put_item(Item=occupants)
            print(response)

        # Print what was just added
        response = dynamodb_client.get_item(
            TableName=TABLE_NAME, Key={"id": {"S": "hoods"}}
        )
        print("Hoods:")
        display(response["Item"])

        response = dynamodb_client.get_item(
            TableName=TABLE_NAME, Key={"id": {"S": "buildings"}}
        )
        print("Buildings:")
        display(response["Item"])

        response = dynamodb_client.get_item(
            TableName=TABLE_NAME, Key={"id": {"S": "labs"}}
        )
        print("Labs:")
        display(response["Item"])

        response = dynamodb_client.get_item(
            TableName=TABLE_NAME, Key={"id": {"S": "occupants"}}
        )
        print("Occupants:")
        display(response["Item"])

In [6]:
dynamodb = boto3.resource('dynamodb', region_name="us-east-1")

table = dynamodb.Table('fumehoods')

response = table.scan(ProjectionExpression="id")
data = response['Items']

# Uncomment for large queries
# while 'LastEvaluatedKey' in response:
#     response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
#     data.extend(response['Items'])

data

[{'id': 'Biotech.Floor_4.Lab_433.Hood_1'},
 {'id': 'BiotechnologyBuilding/lab_433/hood'},
 {'id': 'buildings'},
 {'id': 'cat'},
 {'id': 'Biotech.Floor_3.Lab_317.Hood_1'},
 {'id': 'occupants'},
 {'id': 'maggie'},
 {'id': 'Biotech.Floor_4.Lab_433.Hood_2'},
 {'id': 'hoods'},
 {'id': 'labs'},
 {'id': 'Biotech.Floor_4.Lab_441.Hood_1'}]